In [1]:
import csv
import pandas as pd
import random
import re
import time
from tqdm import tqdm, trange

In [2]:
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df.head()

,paperA,paperB,A_firstAuthorInDegree,A_firstAuthorOutDegree,A_PaperOutDegree,abstractSim,notHasCitationPurpose,ifFirstAuthorInSameAffilaition,ifSameJournal,selfCite,citedPaperJournalTypeProportionInCitingPaperReferences,citedPaperFirstAuthorHasCiteCitingPaperFirstAuthorsPaper,citingPaperAuthorsPapersIntersectionCitedPaperAuthorsPapers,B_firstAuthorInDegree,B_firstAuthorOutDegree,B_PaperOutDegree,label_ifAbnormal
0,2138715703,2007600326,0,171,28,0.637714,0.672244,0,0,0,0.027027,0,0,6,0,0,0
1,2138715703,2147617783,0,171,28,0.569197,0.723836,0,0,0,0.054054,0,0,10,0,0,0
2,2138715703,1966687480,0,171,28,0.473437,0.770526,0,0,0,0.027027,0,0,6,0,0,0
3,2138715703,2062460839,0,171,28,0.572876,0.651287,0,0,0,0.027027,0,0,6,0,0,0
4,2138715703,2134441116,0,171,28,0.524535,0.755068,1,0,0,0.054054,1,1,22,0,0,0


In [3]:
df_t = df.loc[df['label_ifAbnormal'] == 0]
df_f = df.loc[df['label_ifAbnormal'] != 0]

In [4]:
paperaidlist = []
paperaidlist = df_t['paperA'].unique()
paperaidliststr = [str(x) for x in paperaidlist]

In [5]:
paperbidlist = []
paperbidlist = df_t['paperB'].unique()
paperbidliststr = [str(x) for x in paperbidlist]

In [6]:
f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
paperid_abstractdoc2vec = []
for line in f2:
    line = line.strip().split('\t')
    paperid_abstractdoc2vec.append(line[0])

# core paper 和 true cited paper

In [7]:
random.seed(3)
randompaperidliststr = random.sample(paperaidliststr,40)
#have vectors
corepaperidliststr_invec = [x for x in randompaperidliststr if x in paperid_abstractdoc2vec]
print(len(corepaperidliststr_invec))

37


In [8]:
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_corepaperidliststr_invec = []
allpaperidOf_corepaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in corepaperidliststr_invec:
        allpaperidOf_corepaperidliststr_invec.append(line[0])
        alljournalidOf_corepaperidliststr_invec.append(line[10])

172037947it [08:47, 326206.43it/s]


In [9]:
allvalidjournalid = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
for line in tqdm(f) :
    line = line.strip().split('\t')
    allvalidjournalid.append(line[0])
print(len(allvalidjournalid))

7623it [00:00, 965432.07it/s]

7623


In [11]:
#删掉没有journalid的论文
print(len(alljournalidOf_corepaperidliststr_invec))#37
corepaperidliststr = []
for paperid,journalid in zip(allpaperidOf_corepaperidliststr_invec,alljournalidOf_corepaperidliststr_invec):
    if journalid in allvalidjournalid:
        corepaperidliststr.append(paperid)
print(len(corepaperidliststr)) #29
corepaperidlist = [int(x) for x in corepaperidliststr]
print(len(corepaperidlist)) ##29
random.seed(3)
corepaperidlist = random.sample(corepaperidlist,28)
corepaperidliststr = [str(x) for x in corepaperidlist]

37
29
29


In [16]:
#真实被引用的paper，去掉没有 abstract vec的
paperidcitedbycorepaper = []
df_citedpaper = df[df['paperA'].isin(corepaperidlist)]
print(len(df_citedpaper)) #139
paperidcitedbycorepaperori = df_citedpaper['paperB'].unique()
print(len(paperidcitedbycorepaperori)) #135
paperidcitedbycorepaperoristr = [str(x) for x in paperidcitedbycorepaperori]
paperbinabstractem = [x for x in paperidcitedbycorepaperoristr if x in paperid_abstractdoc2vec]
print(len(paperbinabstractem)) #131
paperbinabstractem_uncore = [x for x in paperbinabstractem if x not in corepaperidliststr]
print(len(paperbinabstractem_uncore))#131

139
135
131
131


In [17]:

f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_citedpaperidliststr_invec = []
allpaperidOf_citedpaperidliststr_invec = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in paperbinabstractem_uncore:
        allpaperidOf_citedpaperidliststr_invec.append(line[0])
        alljournalidOf_citedpaperidliststr_invec.append(line[10])
f.close()

172037947it [12:55, 221714.44it/s]


In [18]:
#删掉没有journalid的论文
print(len(alljournalidOf_citedpaperidliststr_invec))#131
citedpaperidliststr = []
for paperid,journalid in zip(allpaperidOf_citedpaperidliststr_invec,alljournalidOf_citedpaperidliststr_invec):
    if journalid in allvalidjournalid:
        citedpaperidliststr.append(paperid)
print(len(citedpaperidliststr)) #113
paperidcitedbycorepaperstr = citedpaperidliststr
print(len(paperidcitedbycorepaperstr))#113
paperidcitedbycorepaper = [int(x) for x in paperidcitedbycorepaperstr]
print(len(paperidcitedbycorepaper)) ##113

131
113
113
113


In [19]:
#删掉core paper中重复的
paperidcitedbycorepaper = sorted(set(paperidcitedbycorepaper))
print(len(paperidcitedbycorepaper)) #113
paperidcitedbycorepaper = [x for x in paperidcitedbycorepaper if x not in corepaperidlist]
print(len(paperidcitedbycorepaper)) #113
paperidcitedbycorepaperstr = [str(x) for x in paperidcitedbycorepaper]
print(len(paperidcitedbycorepaperstr))# 113

113
113
113


# 需要添加的异常引用的paper id+所需要用到的所有paper id

In [21]:
#把core paper分两半，后一半加入异常引用
print(len(corepaperidliststr)) #28
corepaperidliststr1 = corepaperidliststr[0:14]
print(len(corepaperidliststr1))  #14
corepaperidliststr2 = corepaperidliststr[14:28]
print(len(corepaperidliststr2))  #14
corepaperidlist1 = corepaperidlist[0:14]
print(len(corepaperidlist1))  #14
corepaperidlist2 = corepaperidlist[14:28]
print(len(corepaperidlist2))  #14

28
14
14
14
14


In [22]:
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')
alljournalidOf_2nd = []
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[0] in corepaperidliststr2:
        alljournalidOf_2nd.append(line[10])
f.close()

172037947it [07:48, 367203.47it/s]


In [23]:
#delete unvalid journal
annmous_journalstrs = sorted(set(alljournalidOf_2nd))
annmous_journalstrs

['128227336',
 '151741590',
 '171842551',
 '194681035',
 '202381698',
 '203303425',
 '26843219',
 '45281609',
 '49878492',
 '63830647',
 '65139510',
 '70378559',
 '81455967',
 '95464858']

In [27]:
#selected an irrelevant journal
annmous_journalstr = ['ASTRONOMICAL JOURNAL','PUBLICATIONS OF THE ASTRONOMICAL SOCIETY OF THE PACIFIC','ASTROPHYSICAL JOURNAL','ICARUS','PLANETARY AND SPACE SCIENCE','SOLAR SYSTEM RESEARCH','Annual Review of Astronomy and Astrophysics','ASTROPHYSICAL JOURNAL SUPPLEMENT SERIES',
                      'INTERNATIONAL JOURNAL OF MODERN PHYSICS D','ASTROPHYSICS','EXPERIMENTAL ASTRONOMY','ASTROPARTICLE PHYSICS','ASTRONOMY AND ASTROPHYSICS REVIEW','ASTRONOMY REPORTS','ASTRONOMY LETTERS-A JOURNAL OF ASTRONOMY AND SPACE ASTROPHYSICS']

annmous_journalstr = [x.lower() for x in annmous_journalstr]
annmous_journalstr

['astronomical journal',
 'publications of the astronomical society of the pacific',
 'astrophysical journal',
 'icarus',
 'planetary and space science',
 'solar system research',
 'annual review of astronomy and astrophysics',
 'astrophysical journal supplement series',
 'international journal of modern physics d',
 'astrophysics',
 'experimental astronomy',
 'astroparticle physics',
 'astronomy and astrophysics review',
 'astronomy reports',
 'astronomy letters-a journal of astronomy and space astrophysics']

In [44]:
df = pd.read_excel(io='/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/if2017.xlsx')
Subcategory1 = df.loc[df['Subcategory'] == '神经科学']
annmous_journaltitlestr1 = Subcategory1['Full Title'].str.lower()
annmous_journaltitlestr1 = [str(x) for x in annmous_journaltitlestr1]

Subcategory2 = df.loc[df['Subcategory'] == '细胞生物学']
annmous_journaltitlestr2 = Subcategory2['Full Title'].str.lower()
annmous_journaltitlestr2 = [str(x) for x in annmous_journaltitlestr2]

annmous_journaltitlestr = annmous_journaltitlestr1 + annmous_journaltitlestr2

In [45]:
print(len(annmous_journaltitlestr))

138


In [46]:
annmous_journalstr = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[1] in annmous_journaltitlestr:
        annmous_journalstr.append(line[0])
print(len(annmous_journalstr))

7623it [00:00, 393709.88it/s]

131


In [47]:
#papers published by annmous journal
annmous_paperid_ori = []
annmous_journalid_ori = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/Papers.txt','r')

for line in tqdm(f) :
    line = line.strip().split('\t')
    if line[10] in annmous_journalstr:
        annmous_paperid_ori.append(line[0])
        annmous_journalid_ori.append(line[10])
f.close()

172037947it [12:16, 233553.11it/s]


In [48]:
print(len(annmous_paperid_ori))#718303
annmous_paperid0 = sorted(set(annmous_paperid_ori))
print(len(annmous_paperid0))#718303
annmous_paperid1 = [x for x in annmous_paperid0 if x in paperid_abstractdoc2vec]
print(len(annmous_paperid1))#362
annmous_paperid2 = [x for x in annmous_paperid1 if x not in corepaperidliststr]
print(len(annmous_paperid2))#360
annmous_paperid3 = [x for x in annmous_paperid2 if x not in paperidcitedbycorepaperstr]
print(len(annmous_paperid3))#352

718303
718303
362
360
352


In [52]:
copyp = annmous_paperid_ori
copyj = annmous_journalid_ori
print(len(copyp))
random.seed(3)
annmous_paperid = random.sample(annmous_paperid3,112)
annmous_paperidstr = [str(x) for x in annmous_paperid]
print(len(annmous_paperid))

112
112


In [53]:
#所有论文id
allpaperidstr = corepaperidliststr + paperidcitedbycorepaperstr + annmous_paperid
print(len(allpaperidstr)) #253 + 28 + 113 + 112
allpaperid = [int(x) for x in allpaperidstr]
print(len(allpaperid))

253
253


In [54]:

with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/allpaperid.txt', 'w') as cf:
    for line in allpaperidstr:
        cf.write(line+'\n')

cf.close()

# journal和paper-journal

In [55]:
journal = alljournalidOf_corepaperidliststr_invec + alljournalidOf_citedpaperidliststr_invec + annmous_journalid_ori
paper = allpaperidOf_corepaperidliststr_invec + allpaperidOf_citedpaperidliststr_invec + annmous_paperid_ori

alljournalid = []
publishedpaper = []
for item1,item2 in zip(journal,paper):
    if item2 in allpaperidstr:
        alljournalid.append(item1)
        publishedpaper.append(item2)
        
print(len(alljournalid)) #424
#alljournalid = sorted(set(alljournalid))
print(len(publishedpaper)) #424


253
253


In [56]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/paperjournal.txt', 'w') as cf:
    for p,j in zip(publishedpaper,alljournalid):
        cf.write(p + ' ' + j + '\n')

cf.close()

In [57]:
alljournal = sorted(set(alljournalid))
print(len(alljournal))
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/alljournalid.txt', 'w') as cf:
    for line in alljournal:
        cf.write(line+'\n')

cf.close()

124


# author 和author-paper

In [59]:
paperid_inpaperauthor = []
authorid_inpaperauthor = []
f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/PaperAuthorAffiliations.txt','r')

for line in tqdm(f):
    line = line.split('\t')
    if line[0] in allpaperidstr:
        paperid_inpaperauthor.append(line[0]) #str
        authorid_inpaperauthor.append(line[1]) #str    
print(len(paperid_inpaperauthor)) #1634
print(len(authorid_inpaperauthor)) #1634
f.close()

455999370it [32:59, 230374.41it/s]

1634
1634


In [60]:
allauthor = sorted(set(authorid_inpaperauthor))
print(len(allauthor)) #1525
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/allauthorid.txt', 'w') as cf:
    for line in allauthor:
        cf.write(line+'\n')

cf.close()

1525


In [61]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/paperauthor.txt', 'w') as cf:
    for p,a in zip(paperid_inpaperauthor,authorid_inpaperauthor):
        cf.write(p + ' ' + a + '\n')

cf.close()

# citing relationship

In [83]:
citinginfo = df_t.loc[df_t['paperA'].isin(corepaperidlist)]
paperidcitedbycorepaper = [int(x) for x in paperidcitedbycorepaperstr]
citinginfo1 = citinginfo.loc[citinginfo['paperB'].isin(paperidcitedbycorepaper)]
citingpaperid = list(citinginfo1['paperA'])
citedpaperid = list(citinginfo1['paperB'])
    

m = 8

for id1 in corepaperidliststr2:
    paper = []
    i = corepaperidliststr2.index(id1)
    paper = annmous_paperid[i:i+m]
    #print(paper)
    #paper1 = sorted(set(paper))
    paper = [x for x in paper if x != id1]
    for h in paper:
        citingpaperid.append(int(id1))
        citedpaperid.append(h)
        
        
f.close()

In [84]:
print(len(citinginfo1))#真实引用数量 116
print(len(citingpaperid))#总引用数量 228
a = len(citinginfo1)
b = len(citingpaperid)
c = b - a
print(c) #虚假112


116
228
112


In [85]:
with open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/citinginfo.txt', 'w') as cf:
    for ing,ed in zip(citingpaperid, citedpaperid):
        cf.write(str(ing) + ' ' + str(ed) + '\n')

cf.close()

# abstract vec matrix

In [86]:
embeddingvectors = []
x = 0
print(len(allpaperidstr))#253
for ids in allpaperidstr:
    f2 = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/allPaperAbstracts-doc2vec-Embedding-128.txt','r')
    for line in f2:
        line = line.strip().split('\t')
        if line[0] == ids:
            x+=1
            vectors = line[1].split(' ')

            embeddingvectors.append(vectors[0:129]) #str
            print(len(vectors))
            
f2.close()
print(x)#253

253
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


In [87]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/abstract2vec.txt","w")
 
for line in embeddingvectors:
    for item in line:
        f.write(item + ' ')
    f.write('\n')
f.close()

# journal IF matrix

In [88]:
df = pd.read_excel(io='/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/if2017.xlsx')
Full_Title = df['Full Title'].str.lower()
IF = df['IF 2016-2017']
Full_Title = list(Full_Title)
IF = list(IF) #102

alljournalnames = []
print(len(alljournal)) #124
for jj in alljournal:
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/coauthor/processed/selectedJournals.txt','r')
    for line in tqdm(f) :
        line = line.strip().split('\t')
        if line[0] == jj:
            alljournalnames.append(line[1])
print(len(alljournalnames)) #124

IF_journal = []
for name in alljournalnames:
    for title,ifs in zip(Full_Title,IF):
        if title == name:
            i = Full_Title.index(title)
            score = IF[i]
            IF_journal.append(score)
print(len(IF_journal)) #124

124


7623it [00:00, 1012610.59it/s]
7623it [00:00, 915245.30it/s]
7623it [00:00, 976667.97it/s]
7623it [00:00, 984608.12it/s]
7623it [00:00, 820203.67it/s]
7623it [00:00, 838530.80it/s]
7623it [00:00, 1108140.55it/s]
7623it [00:00, 865543.57it/s]
7623it [00:00, 818523.87it/s]
7623it [00:00, 1125023.91it/s]
7623it [00:00, 934423.81it/s]
7623it [00:00, 1109140.02it/s]
7623it [00:00, 841864.70it/s]
7623it [00:00, 1036878.30it/s]
7623it [00:00, 1082808.84it/s]
7623it [00:00, 904065.47it/s]
7623it [00:00, 1134404.09it/s]
7623it [00:00, 891163.93it/s]
7623it [00:00, 979330.42it/s]
7623it [00:00, 1126490.48it/s]
7623it [00:00, 919773.87it/s]
7623it [00:00, 1146485.20it/s]
7623it [00:00, 884752.32it/s]
7623it [00:00, 1067944.13it/s]
7623it [00:00, 967272.10it/s]
7623it [00:00, 946736.33it/s]
7623it [00:00, 1082332.33it/s]
7623it [00:00, 791336.98it/s]
7623it [00:00, 979540.44it/s]
7623it [00:00, 1042796.37it/s]
7623it [00:00, 875521.77it/s]
7623it [00:00, 1031925.49it/s]
7623it [00:00, 954195.40it/

124
124


In [89]:

f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/IFS.txt","w")
 
for line in IF_journal:
    f.write(str(line))
    f.write('\n')
f.close()

# author credit matrix

In [90]:
#author credit
df = pd.read_csv('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/inputTable.csv')
df_annomous = df.loc[df['label_ifAbnormal'] == 1]
dfa = []
dfb = []
dfa = df_annomous['paperA'].unique()
dfb = df_annomous['paperB'].unique()
creditpapers = []
creditpapers = list(dfa) + list(dfb) #int
creditpapers = sorted(set(creditpapers),key = creditpapers.index)
creditpaperstr = [str(x) for x in creditpapers]

In [95]:
annmousauthors = []
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/annmousauthors.txt","r")
for line in tqdm(f):
   
    annmousauthors.append(line)

f.close()
print(len(annmousauthors))  #2014

2014it [00:00, 1133413.16it/s]

2014


In [96]:
annmousauthorslist = sorted(set(annmousauthors))
print(len(annmousauthorslist)) #1779

1779


In [98]:
matrixcredic = [[1 for i in range(2)] for j in range(len(allauthor))]
#matrixcredic.shape
i = 0
h = 0
for i in range(len(allauthor)):
    matrixcredic[i][0] = allauthor[i]
    i += 1
for item in matrixcredic:
    if item[0] in annmousauthors:
        item[1] = 0
        h += 1
credit = []
for item in matrixcredic:
    credit.append(item[1])#int
print(len(credit)) #2566
print(h) #76

1525
0


In [99]:
creditstr = [str(x) for x in credit]
f=open("/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/authorcredit.txt","w")
 
for line in creditstr:
    f.write(line)
    f.write('\n')
f.close()

# journal paper matrix

In [100]:
journalpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(alljournal))]

for ids,js in zip(allpaperidstr,alljournal):
    j = allpaperidstr.index(ids)
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/paperjournal.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == ids:
            i = alljournal.index(js)
            journalpapermatrix[i][j] = 1

In [101]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/journalpaper.txt","w")
 
for line in journalpapermatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author author matrix

In [102]:
coauthormatrix = [[0 for i in range(len(allauthor))] for j in range(len(allauthor))]
for paperid in allpaperidstr:
    authors = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if line[0] == paperid:
            authors.append(line[1])
    authors = sorted(set(authors))
    for author1 in authors:
        i = allauthor.index(author1)
        for author2 in authors:
            j = allauthor.index(author2)
            coauthormatrix[i][j] += 1

In [103]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/authorauthor.txt","w")
 
for line in coauthormatrix:
    for item in line:
        f.write(str(item)+ ' ')
    f.write('\n')
f.close()

# author paper matrix

In [104]:
print(len(citingpaperid)) #228
print(len(citedpaperid))#228
print(len(citinginfo1))#116

228
228
116


In [106]:
authorpapermatrix = [[0 for i in range(len(allpaperidstr))] for j in range(len(allauthor))]
allpaperid = [int(x) for x in allpaperid]
for a,b in zip(citingpaperid,citedpaperid):
    j = allpaperid.index(int(b))
    author = []
    f = open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/processed/paperauthor.txt', 'r')
    for line in f:
        line = line.strip().split(' ')
        if int(line[0]) == a:
            author.append(line[1])
    author = sorted(set(author))
    for au in author:
        i = allauthor.index(au)
        authorpapermatrix[i][j] = 1
    
f.close()

In [107]:
f=open(r"/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/irrelevant/matrix/authorpaper.txt","w")
 
for line in tqdm(authorpapermatrix):
    for item in line:
        f.write(str(item)+' ')
    f.write('\n')
f.close()

100%|████████████████████████████████████| 1525/1525 [00:00<00:00, 10449.36it/s]


# 计算引用链接数量和合作链接数量

In [108]:
sum(map(sum,authorpapermatrix)) #引用链接数量

1172

In [109]:
sum(map(sum,coauthormatrix))#合作链接数量

17232

In [110]:
s = 0
for i in range(len(allauthor)):
    s += coauthormatrix[i][i]
    i += 1
print(s)

h = (sum(map(sum,coauthormatrix)) - s)/2
print(h)

1634
7799.0
